In [ ]:
 %pip install -U semantic-link 

In [ ]:
%pip install semantic-link-labs

# Import necessary libraries

In [ ]:
import sempy.fabric as fabric
import sempy_labs as labs
import pandas as pd 


# List all workspaces that are on dedicated capacity

In [ ]:
workspaces = fabric.list_workspaces().query('`Is On Dedicated Capacity` == True')

# Display the contents of fabric.list_workspaces()
display(workspaces)

# Concatenate items from the  workspaces, adding the workspace name to each row

In [ ]:
df_list = []
for ws in workspaces.itertuples(index=False):
    items_df = fabric.list_items(workspace=ws.Id)
    items_df['WorkspaceName'] = ws.Name  # Add the workspace name column
    df_list.append(items_df)

df = pd.concat(df_list, ignore_index=True)


In [ ]:
display (df)

# Rename 'Display Name' column to 'DatasetName'

In [ ]:
df.rename(columns={'Display Name': 'DatasetName'}, inplace=True)

# Filter the dataframe to include only rows where the type is 'SemanticModel'

In [ ]:
semantic_model_df = df[df['Type'] == 'SemanticModel']


In [ ]:
display(semantic_model_df)


# Create an empty dataframe to store all BPA results

In [ ]:

all_bpa_results = pd.DataFrame()

# Run the BPA for each dataset and workspace, and collect the results

In [ ]:
for row in semantic_model_df.itertuples(index=False):
    dataset_name = row.DatasetName
    workspace_name = row.WorkspaceName
    try:
        bpa_result = labs.run_model_bpa(dataset=dataset_name, workspace=workspace_name, return_dataframe=True)
        
        bpa_result['WorkspaceName'] = workspace_name
        bpa_result['DatasetName'] = dataset_name
        
        
        all_bpa_results = pd.concat([all_bpa_results, bpa_result], ignore_index=True)
    except Exception as e:
        print(f"Error processing dataset '{dataset_name}' in workspace '{workspace_name}': {e}")

# Save the BPA results to a lakehouse table

In [ ]:

lakehouse_name = 'Governance'
delta_table_name = 'bpa_results_table'
labs.save_as_delta_table(dataframe=all_bpa_results, delta_table_name=delta_table_name, write_mode='overwrite', lakehouse=lakehouse_name)


display(all_bpa_results)
